In [483]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
import time

In [501]:
m = 10000
terms_no = 11
rnn_units = 20
a = np.array([.6, .5, -.2][::-1])
rcr_depth = a.size
X = np.random.random((m,3))
cols = X.shape[1]
for i in range(cols, terms_no):
    ut = np.random.random((m,1))*.1 * 0
    X = np.append(X, X[:,-rcr_depth:].dot(a)[:, np.newaxis]+ut, axis=1)
X = X.reshape(m, terms_no, 1)
timesteps = terms_no-1

In [502]:
tr_X, te_X, tr_y, te_y = train_test_split(X[:,:-1,[-1]], X[:,1:,[-1]], random_state=442, test_size=.2)

In [503]:
#tr_X, te_X, tr_y, te_y = train_test_split(X[:,:-1], X[:,[-1]], random_state=442, test_size=.2)

In [504]:
run = 0

In [505]:
run += 1
tf.reset_default_graph()
with tf.device('/cpu:0'):
    t_X = tf.placeholder(X.dtype, (None, timesteps, 1), 'input')
    t_y = tf.placeholder(X.dtype, (None, timesteps, 1), 'target')
#     x2sq = tf.unstack(t_X, timesteps, 1)
#     y2sq = tf.unstack(t_y, timesteps, 1)
    
    RNN_cell = tf.contrib.rnn.BasicRNNCell(num_units=rnn_units) #, activation=tf.nn.relu)
    rnn_o, state = tf.nn.dynamic_rnn(RNN_cell, t_X, dtype=X.dtype)
    flat = tf.reshape(rnn_o, [-1, rnn_units])
    dense = tf.layers.dense(flat, 1)
    output = tf.reshape(dense, [-1, timesteps, 1])
    loss = tf.reduce_mean(tf.square(tf.subtract(t_y, output)))
    opt = tf.train.AdamOptimizer(learning_rate=.01).minimize(loss)
#     init_state = tf.zeros(RNN_l.state_size, dtype=np.float64)
#     output = RNN_l(t_X, init_state)
tf.summary.scalar('loss', loss)
writer = tf.summary.FileWriter('.log/run_{}'.format(run))
merged_smr = tf.summary.merge_all()

In [506]:
tm = time.time()


epochs = 200
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(epochs):
        smr = sess.run(merged_smr, feed_dict={t_X: tr_X, t_y: tr_y})
        writer.add_summary(smr, i)
        sess.run(opt, feed_dict={t_X: tr_X, t_y: tr_y})
    rslt = sess.run(output, feed_dict={t_X: te_X})
writer.close()

print('time taken =', time.time()-tm)

time taken = 46.305506467819214


In [507]:
np.insert(rslt[:,-1,[0]], [0], te_y[:,-1,[0]], axis=1)[:30]

array([[ 0.13009607,  0.12884938],
       [ 0.0238374 ,  0.0243486 ],
       [ 0.04641526,  0.04655952],
       [ 0.15592102,  0.15637569],
       [-0.01800432, -0.01447365],
       [ 0.44751766,  0.44470416],
       [ 0.18739583,  0.18669188],
       [ 0.11424729,  0.11296298],
       [ 0.31291922,  0.3110876 ],
       [ 0.45107271,  0.44768853],
       [ 0.305316  ,  0.30591191],
       [ 0.02753778,  0.02847668],
       [ 0.20080299,  0.19989825],
       [ 0.28766737,  0.28814277],
       [ 0.28420331,  0.28485416],
       [ 0.11779271,  0.11735892],
       [ 0.34999164,  0.34825939],
       [ 0.09510082,  0.0951478 ],
       [ 0.45283154,  0.44940535],
       [ 0.06747935,  0.06846034],
       [ 0.32120002,  0.31932869],
       [ 0.03912514,  0.03916878],
       [ 0.2323829 ,  0.23128713],
       [ 0.28262574,  0.28082225],
       [ 0.11794063,  0.11973414],
       [ 0.09810441,  0.09797827],
       [ 0.23100026,  0.22942609],
       [ 0.25863959,  0.25712182],
       [ 0.27137783,

In [509]:
delta = np.abs(rslt[:,-1,[0]] - te_y[:,-1,[0]])
print('Percentage of errors is', delta[np.where(delta>.01)].shape[0]/delta.shape[0] * 100)

Percentage of errors is 0.0
